#### Imports

In [1]:
# Telegram Bot
import telebot
import difflib
from telebot.types import InlineKeyboardMarkup, InlineKeyboardButton
from datetime import datetime

# Import provisorios
from ingressos.ibge import Ibge
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Config Driver
from config.config_driver import ConfigDriver

# Definitions
from definitions import *

['Adamantina', 'Adolfo', 'Aguaí', 'Águas da Prata', 'Águas de Lindóia', 'Águas de Santa Bárbara', 'Águas de São Pedro', 'Agudos', 'Alambari', 'Alfredo Marcondes', 'Altair', 'Altinópolis', 'Alto Alegre', 'Alumínio', 'Álvares Florence', 'Álvares Machado', 'Álvaro de Carvalho', 'Alvinlândia', 'Americana', 'Américo Brasiliense', 'Américo de Campos', 'Amparo', 'Analândia', 'Andradina', 'Angatuba', 'Anhembi', 'Anhumas', 'Aparecida', "Aparecida d'Oeste", 'Apiaí', 'Araçariguama', 'Araçatuba', 'Araçoiaba da Serra', 'Aramina', 'Arandu', 'Arapeí', 'Araraquara', 'Araras', 'Arco-Íris', 'Arealva', 'Areias', 'Areiópolis', 'Ariranha', 'Artur Nogueira', 'Arujá', 'Aspásia', 'Assis', 'Atibaia', 'Auriflama', 'Avaí', 'Avanhandava', 'Avaré', 'Bady Bassitt', 'Balbinos', 'Bálsamo', 'Bananal', 'Barão de Antonina', 'Barbosa', 'Bariri', 'Barra Bonita', 'Barra do Chapéu', 'Barra do Turvo', 'Barretos', 'Barrinha', 'Barueri', 'Bastos', 'Batatais', 'Bauru', 'Bebedouro', 'Bento de Abreu', 'Bernardino de Campos', 'Ber

#### Variaveis

In [2]:
API_TOKEN = '7135171388:AAExkHE82hR68W21q-KJCsxpJmVU9gtLNPA'
bot = telebot.TeleBot(API_TOKEN)

brazilian_states = ["Acre", "Alagoas", "Amapá", "Amazonas", "Bahia", "Ceará", "Distrito Federal", 
                    "Espírito Santo", "Goiás", "Maranhão", "Mato Grosso", "Mato Grosso do Sul", 
                    "Minas Gerais", "Pará", "Paraíba", "Paraná", "Pernambuco", "Piauí", 
                    "Rio de Janeiro", "Rio Grande do Norte", "Rio Grande do Sul", "Rondônia", 
                    "Roraima", "Santa Catarina", "São Paulo", "Sergipe", "Tocantins"]

config_driver = ConfigDriver()
driver = config_driver.get_driver()


#### Main

In [3]:
cinemas = None

@bot.message_handler(func=lambda message: True)
def reply_message(message):
    #Apenas para cinema
    if message.text == "/start":
        handle_first_contact(message)

def handle_first_contact(message):
    markup = InlineKeyboardMarkup()
    markup.row_width = 2
    markup.add(InlineKeyboardButton("Cinema", callback_data="cinema"),
               InlineKeyboardButton("Filme", callback_data="filme"))
    bot.send_message(message.chat.id, "Olá, eu sou o bot do Ingresso.com, deseja buscar por:", reply_markup=markup)

#Inicia o processo de busca de cinemas
def handle_cinema_markup(message):
    # Recebe o estado enviado
    state_msg = bot.send_message(message.chat.id, "Digite o estado do cinema que deseja buscar")
    bot.register_next_step_handler(state_msg, process_state_step)

#Processa o estado informado
def process_state_step(message):
    state = message.text
    closest_matches = difflib.get_close_matches(state, brazilian_states, n=1, cutoff=0.6)
    if closest_matches:
        process_city_step(message, closest_matches[0])
    else:
        bot.send_message(message.chat.id, "Nenhum estado correspondente encontrado. Tente novamente.")
        handle_cinema_markup(message)


def process_city_step(message, state):
    city_msg = bot.send_message(message.chat.id, "Agora digite a sua cidade")
    bot.register_next_step_handler(city_msg, lambda msg: process_city_name(msg, state))


def process_city_name(message, state):
    city = message.text
    
    # Instancia a classe do Ibge
    ibge = Ibge()
    
    # Obtem o Id do estado
    stateId = ibge.estados[state]
    
    # Obtem as cidades do estado
    citys_from_state = ibge.obter_cidades_por_estado(stateId)
    
    # Verifica se a cidade informada está no estado informado
    closest_matches = difflib.get_close_matches(city, citys_from_state, n=1, cutoff=0.6)
    
    # Se achar a cidade informa os cinemas da cidade
    if closest_matches:
        city = closest_matches[0]
        bot.send_message(message.chat.id, f"Buscando cinemas em {city}...")
        
        formatted_city = city.replace(" ", "-").lower()
        
        linkWebDriver = f"https://www.ingresso.com/cinemas?city={formatted_city}"
        
        driver.get(linkWebDriver)
        
        try:
            fecharPopup = WebDriverWait(driver, 99).until(
                EC.presence_of_element_located((By.CLASS_NAME, "accept-btn"))
            )
            if(fecharPopup.is_displayed()):
                fecharPopup.click()
        except Exception:
            print("Popup button not found, continuing without clicking.")
        
        cinemas = getCinemasFromCity(city, driver, message)
    # Se não achar ele repete esse passo
    else:
        bot.send_message(message.chat.id, "Nenhuma cidade correspondente encontrada. Tente novamente.")
        process_city_step(message, state)
    
        
def getCinemasFromCity(self, driver , message):
    
    try:
        fecharPopup = WebDriverWait(driver, 99).until(
            EC.presence_of_element_located((By.CLASS_NAME, "accept-btn"))
        )
        if(fecharPopup.is_displayed()):
            fecharPopup.click()
    except Exception:
        print("Popup button not found, continuing without clicking.")
    
    global cinemas 
    cinemas = driver.find_elements(By.CSS_SELECTOR, "h3.text-base.leading-4")  

    # Print the names of the cinemas for debugging purposes
    for cinema in cinemas:
        print(cinema.text)

    # Criar um teclado inline
    markup = InlineKeyboardMarkup()

    # Adicionar botões ao teclado com o texto do cinema e dados de callback
    for index, cinema in enumerate(cinemas):
        markup.add(InlineKeyboardButton(cinema.text, callback_data="PORRA_" + str(index)))
    # Enviar o teclado inline para o usuário
    bot.send_message(message.chat.id, "Cinemas:", reply_markup=markup)

            
def getFilmesByCinema(driver, cinemaSelecionado):    
    global cinemas
    
    try:
        fecharPopup = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "accept-btn"))
        )
        if(fecharPopup.is_displayed()):
            fecharPopup.click()
    except Exception:
        print("Popup button not found, continuing without clicking.")
    
    # Seleciona o primeiro cinema (pode ser ajustado)
    cinema_element = cinemas[cinemaSelecionado].find_element(By.XPATH, '..')

    # Rolagem para o cinema selecionado
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", cinema_element)
    
    time.sleep(2)
    # Espera até que o elemento esteja visível e clicável
    WebDriverWait(driver, 10).until(EC.visibility_of(cinema_element))
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable(cinema_element))

    cinema_element.click()

    # Obter a data atual
    today = datetime.today()

    # Formatar a data no formato "dd/MM"
    # data = today.strftime("%d/%m")
    
    data = "28/09"
    
    try:
        diaEscolhidoElemento = driver.find_element(By.XPATH, f"//span[contains(text(), '{data}')]")
    except Exception:
        print(f"Element not found for date: {data}")
        # Optionally, you can retry after a delay
        time.sleep(2)
        try:
            diaEscolhidoElemento = driver.find_element(By.XPATH, f"//span[contains(text(), '{data}')]")
        except Exception:
            print(f"Element still not found for date: {data} after retrying")
            # Handle the failure case as needed
    
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", diaEscolhidoElemento)
    
    # Close the cookie consent if it exists
    try:
        cookie_consent = driver.find_element(By.CLASS_NAME, "CookieConsent")
        if cookie_consent.is_displayed():
            close_button = cookie_consent.find_element(By.TAG_NAME, "button")
            close_button.click()
    except Exception:
        print("Cookie consent not found, continuing without clicking.")

    # Now attempt to click the desired element
    try:
        diaEscolhidoElemento = driver.find_element(By.XPATH, f"//span[contains(text(), '{data}')]")
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", diaEscolhidoElemento)
        diaEscolhidoElemento.click()
    except Exception:
        print("Element click intercepted, please check the page layout.")

    # Captura os blocos de filmes
    blocos_filmes = driver.find_elements(By.CSS_SELECTOR, "relative my-5  bg-ing-neutral-600 p-4")
    print("PASSSEI AQUI")
    print(blocos_filmes)
    # Itera sobre cada bloco para extrair o nome do filme, horários e links
    for bloco in blocos_filmes:
        # Extrai o nome do filme
        nome_filme = bloco.find_element(By.CSS_SELECTOR, "h3.line-clamp-3 a").text
        
        # Extrai os elementos <a> que contêm os horários
        horarios_elements = bloco.find_elements(By.CSS_SELECTOR, "a[href*='sessionId']")
        
        # Extrai o horário e o link
        horarios = []
        for horario_element in horarios_elements:
            horario = horario_element.find_element(By.CSS_SELECTOR, "span").text  # Extrai o texto do horário
            link = horario_element.get_attribute("href")  # Extrai o link
            horarios.append((horario, link))  # Adiciona à lista de horários

        # Exibe os resultados
        print(f"Nome do Filme: {nome_filme}")
        for horario, link in horarios:
            print(f"Horário: {horario} - Link: {link}")
        print("---")
        

@bot.callback_query_handler(func=lambda call: True)
def callback_query(call):
    if call.data == "cinema":
        handle_cinema_markup(call.message)
    elif call.data == "filme":
        bot.send_message(call.message.chat.id, "Essa função ainda não está disponível")
    elif call.data.startswith("PORRA"):
        bot.send_message(call.message.chat.id, call.data)
        index = int(call.data.split("_")[1])
        getFilmesByCinema(driver, index)

#### Run Bot

In [4]:
bot.polling()

Cinemark Center Vale
Cinemark Colinas
Cinépolis Jardim Oriente
Kinoplex Diamante
Kinoplex Vale Sul
Element not found for date: 28/09
Element click intercepted, please check the page layout.
PASSSEI AQUI
[]
